In [3]:
!pip install selenium


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip3 install webdriver_manager


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# selenium: 웹 브라우저를 이용, 웹 사이트 제어를 위해 사용
# bs4: html에서 데이터 추출, html 및 xml 문서를 parsing 하기 위해 사용

### 네이버 지도 카페 상세 정보

In [6]:
import json
import time
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [7]:
os.getcwd()

'/Users/yangchaeyeon/Documents'

In [28]:
# 네이버 지도에서 크롤링
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument("/Users/yangchaeyeon/Downloads/chromedriver-mac-arm64")
driver = webdriver.Chrome()
driver.get("https://map.naver.com/p")
key_word = ''
print(driver.title)

네이버 지도


In [29]:
# css 찾을 떄까지 10초 대기
def time_wait(num, code):
  try:
    wait = None  # 초기화
    wait = WebDriverWait(driver, num).until(
      EC.presence_of_all_elements_located((By.CSS_SELECTOR, code)))
  except: 
    print(code, "태그를 찾지 못했습니다.")
    driver.quit()
  return wait

# frame 변경 메소드
def switch_frame(frame):
  driver.switch_to.default_content() # frame 초기화
  driver.switch_to.frame(frame)
  

# 페이지 다운
def page_down(num):
  body = driver.find_element(By.CSS_SELECTOR, 'body')
  body.click(driver)
  for i in range(num):
    body.send_keys(Keys.PAGE_DOWN)
    
    
# css를 찾을 때까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

[<selenium.webdriver.remote.webelement.WebElement (session="219e0e378224c94a7e19f72364ea1409", element="1983A189EF718BF7465EC59AF607403C_element_41")>]

In [30]:
# 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
key_word = '보연희'
search.send_keys(key_word) # 검색어 입력
search.send_keys(Keys.ENTER) # 엔터버튼 누르기

sleep(1)

In [31]:
# frame 변경
switch_frame(driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe"))

In [32]:
# 카페 방문리뷰수
cafe_visit = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a')

In [33]:
# 카페 블로그리뷰수
cafe_blog = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a')

In [34]:
# 숫자만 가져오기
import re
visit_count = re.search(r'\d+', cafe_visit.text).group()
blog_count = re.search(r'\d+', cafe_blog.text).group()
print('방문자 수', str(visit_count))
print('블로그 수', str(blog_count))

방문자 수 464
블로그 수 286


In [35]:
# 카페 주소
address = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/a/span[1]').text
print(address)

서울 서대문구 연희맛로 17-63 2층


In [36]:
# 카페 고유 ID
ID_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a')
href = ID_element.get_attribute('href')
cafe_ID = re.search(r'\d+', href).group()

print(cafe_ID)

1677855640


In [37]:
import pandas as pd

data = [{
        '카페ID': cafe_ID,
        '카페이름': key_word,
        '방문자리뷰': visit_count,
        '블로그리뷰': blog_count,
        '주소': address
        }]

df = pd.DataFrame(data)
df

,카페ID,카페이름,방문자리뷰,블로그리뷰,주소
0,1677855640,보연희,464,286,서울 서대문구 연희맛로 17-63 2층
